In [ ]:
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
from keras.layers import Flatten, Dense, GlobalAveragePooling2D,Dropout,Conv2D
from keras.applications.densenet import DenseNet169, DenseNet121
from keras.applications.xception import Xception
from keras.models import Model
import keras.backend as K
from sklearn.metrics import roc_auc_score, cohen_kappa_score
import cv2
import os
import random
from sklearn.utils import shuffle
from keras.applications.mobilenetv2 import MobileNetV2
from keras.callbacks import Callback,ModelCheckpoint
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.losses import binary_crossentropy
from keras.utils import Sequence
from keras import backend as keras
from keras.preprocessing.image import ImageDataGenerator
from glob import glob
from tqdm import tqdm
import csv
import math
from keras.callbacks import LearningRateScheduler
from keras.layers import BatchNormalization as bn
%matplotlib inline

In [ ]:
#@title Unnecessary callbacks
from keras.callbacks import Callback
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score

class Metrics(Callback):
    def on_train_begin(self, logs={}):
     self.val_f1s = []
     self.val_recalls = []
     self.val_precisions = []

    def on_epoch_end(self, epoch, logs={}):
     val_predict = (np.asarray(self.model.predict(X_test_feats))).round()
     val_targ = Y_test
#      print(val_targ.shape)   
#      print(val_targ)
#      print(val_predict.shape)   
#      print(val_predict)   
     targ = []
     pred = []
     for x in val_targ:
        targ.append(np.argmax(x))
     for x in val_predict:
        pred.append(np.argmax(x))
#      print(targ)
#      print(pred)
        
     _val_f1 = f1_score(targ, pred)
     
     _val_recall = recall_score(targ, pred)
     _val_precision = precision_score(targ, pred)
     self.val_f1s.append(_val_f1)
     self.val_recalls.append(_val_recall)
     self.val_precisions.append(_val_precision)
     print("— val_f1: {0} — val_precision: {1} — val_recall {2} ".format(_val_f1, _val_precision, _val_recall))
     return

metrics = Metrics()

In [ ]:
#@title Good to have callbacks
from sklearn.metrics import roc_auc_score
from keras.callbacks import Callback
class roc_callback(Callback):
    def __init__(self,validation_data):
#         self.x = training_data[0]
#         self.y = training_data[1]
        self.x_val = validation_data[0]
        self.y_val = validation_data[1]


    def on_train_begin(self, logs={}):
        return

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):
#         y_pred = self.model.predict(self.x)
#         roc = roc_auc_score(self.y, y_pred)
        if(epoch%1==0):
            print("Calc Roc")
            all_rocs = []
            y_pred_val = self.model.predict(X_test,verbose=1)
            #print ("Intitial y_pred:{0}".format(y_pred_val))
            for d in range(2):
                try:
                    roc_val = roc_auc_score(Y_test[:,d], y_pred_val[:,d])
                    all_rocs.append(roc_val)
                except:
                    pass
            all_rocs = np.array(all_rocs)
            mean_roc = np.mean(all_rocs)
            
            Y_test_kappa=[np.argmax(i) for i in Y_test]
            y_pred_kappa=[np.argmax(i) for i in y_pred_val]
            try:
                score = f1_score(Y_test_kappa, np.around(y_pred_kappa))
                f1s = score
            except:
                    print(global_cls[d])
            avg_f1 = f1s
            
            kappa_score = cohen_kappa_score(Y_test_kappa, y_pred_kappa)            
            
            print("ROC VAL {0}".format(mean_roc))
            print("F1 {0}".format(avg_f1))
            print("KAPPA {0}".format(kappa_score))
        return

    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        return

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!ls "/content/gdrive/My Drive/Colab Data/"

In [ ]:
X_train_initial=np.load('/content/gdrive/My Drive/Colab Data/x_train_final(1).npy', mmap_mode='r+')
#Y_train_initial=np.load('/content/gdrive/My Drive/Colab Data/y_train_int8_1channel_segmentation.npy', mmap_mode='r+')
#X_test_inital=np.load('/content/gdrive/My Drive/Colab Data/x_val_final.npy', mmap_mode='r+')
#Y_test_initial=np.load('/content/gdrive/My Drive/Colab Data/y_val_int8_1channel_segmentation.npy', mmap_mode='r+')

In [ ]:
X_train=[]
for index, image in enumerate(X_train_initial):
  X_train.append(cv2.resize(image, (768,768)))
  print (index)

In [ ]:
del X_train_initial

In [ ]:
X_train=np.array(X_train)

In [ ]:
np.save()

In [ ]:
#@title reshape arrays
Y_train=np.reshape(Y_train,(1905,768,768,1))
Y_test=np.reshape(Y_test, (474,768,768,1))

In [ ]:
#@title test array
X_test=np.load('/content/gdrive/My Drive/Colab Data/X_test.npy', mmap_mode='r+')

In [ ]:
#@title Train arrays
X_train=[]
Y_train=[]
for index,image in enumerate(Y_train_initial):
  if np.max(image)==0:
    print ("Healthy")
  else:
    b=cv2.resize(X_train_initial[index], (768,768))
    X_train.append(b)
    a=cv2.resize(Y_train_initial[index], (768,768))
    Y_train.append(a)
  print (index)

del X_train_initial
del Y_train_initial

In [ ]:
#@title Save numpy
np.save('/content/gdrive/My Drive/Colab Data/X_test_pneumo_768.npy', X_test)
np.save('/content/gdrive/My Drive/Colab Data/Y_test_pneumo_768.npy', Y_test)

In [ ]:
#@title Test array
X_test=[]
Y_test=[]
for index,image in enumerate(Y_test_initial):
  if np.max(image)==0:
    print ("Healthy")
  else:
    b=cv2.resize(X_test_initial[index], (768,768))
    X_test.append(b)
    a=cv2.resize(Y_test_initial[index], (768,768))
    Y_test.append(a)
  print (index)

del X_test_initial
del Y_test_initial

In [ ]:
#@title Default title text
X_train=np.array(X_train)
X_test=np.array(X_test)
Y_train=np.array(Y_train)
Y_test=np.array(Y_test)

In [ ]:
#@title Division by 255
Y_test=Y_test/255
Y_train=Y_train/255

In [ ]:
#@title Save arrays
np.save('/content/gdrive/My Drive/Colab Data/Y_train_pneumo_768.npy', Y_train)
np.save('/content/gdrive/My Drive/Colab Data/Y_test_pneumo_768.npy', Y_test)

In [ ]:
#@title Segmentation model
def dice_coef(y_true, y_pred):
    y_true_f = keras.flatten(y_true)
    y_pred_f = keras.flatten(y_pred)
    intersection = keras.sum(y_true_f * y_pred_f)
    return (2. * intersection + 1) / (keras.sum(y_true_f) + keras.sum(y_pred_f) + 1)

def dice_coef_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)

segmentation_model = load_model('/content/gdrive/My Drive/Colab Data/Weights/unet_lung_seg.hdf5', \
                                custom_objects={'dice_coef_loss': dice_coef_loss, \
                                                'dice_coef': dice_coef})

segmentation_model.summary()

# Train the unet with segmentation

In [ ]:
class dice_call(Callback):
    def __init__(self,validation_data):
        self.x_val = validation_data[0]
        self.y_val = validation_data[1]

    def on_epoch_end(self, epoch, logs={}):

        if(epoch%1==0):
            print("Calc DICE")
            y_pred_val = np.around(self.model.predict(self.x_val,verbose=1))
            try:
                dice_val = dice(self.y_val, y_pred_val)
            except:
                pass
            print("DICE VAL {0}".format(dice_val))

        return

In [ ]:
dice_call1 = dice_call((X_test,Y_test))

In [ ]:
def dice(true, preds):  
    scores  = []
    for i in range(len(true)):
        im1 = true[i]
        im2 = preds[i]
        intersection = np.logical_and(im1, im2)
        score = 2. * intersection.sum() / (im1.sum() + im2.sum())
        scores.append(score)
    return np.array(scores).mean()#, scores

def dice_score(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + K.epsilon()) / (K.sum(y_true_f) + K.sum(y_pred_f)+K.epsilon())

def dice_loss(y_true, y_pred):
    return 1 - dice_score(y_true, y_pred)

In [ ]:
smooth = 1.0
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f1 = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f1)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f1) + smooth)

In [ ]:
#@title Old U_net
def u_net(input_shape, dropout_rate, l2_lambda):
 
    # Encoder
    input = Input(shape = input_shape, name = "input")
    conv1_1 = Conv2D(32, (3, 3), padding = "same", activation='relu', kernel_regularizer=regularizers.l2(l2_lambda), name = "conv1_1")(input)
    conv1_1 = bn(name = "conv1_1_bn")(conv1_1)
    conv1_2 = Conv2D(32, (3, 3), padding = "same", activation='relu', kernel_regularizer=regularizers.l2(l2_lambda), name = "conv1_2")(conv1_1)
    conv1_2 = bn(name = "conv1_2_bn")(conv1_2)
    pool1 = MaxPooling2D(name = "pool1")(conv1_2)
    drop1 = Dropout(dropout_rate)(pool1)

    conv2_1 = Conv2D(64, (3, 3), padding = "same", activation='relu', kernel_regularizer=regularizers.l2(l2_lambda), name = "conv2_1")(pool1)
    conv2_1 = bn(name = "conv2_1_bn")(conv2_1)
    conv2_2 = Conv2D(64, (3, 3), padding = "same", activation='relu', kernel_regularizer=regularizers.l2(l2_lambda), name = "conv2_2")(conv2_1)
    conv2_2 = bn(name = "conv2_2_bn")(conv2_2)
    pool2 = MaxPooling2D(name = "pool2")(conv2_2)
    drop2 = Dropout(dropout_rate)(pool2)

    conv3_1 = Conv2D(128, (3, 3), padding = "same", activation='relu', kernel_regularizer=regularizers.l2(l2_lambda), name = "conv3_1")(pool2)
    conv3_1 = bn(name = "conv3_1_bn")(conv3_1)
    conv3_2 = Conv2D(128, (3, 3), padding = "same", activation='relu', kernel_regularizer=regularizers.l2(l2_lambda), name = "conv3_2")(conv3_1)
    conv3_2 = bn(name = "conv3_2_bn")(conv3_2)
    pool3 = MaxPooling2D(name = "pool3")(conv3_2)
    drop3 = Dropout(dropout_rate)(pool3)  

    conv4_1 = Conv2D(256, (3, 3), padding = "same", activation='relu', kernel_regularizer=regularizers.l2(l2_lambda), name = "conv4_1")(pool3)
    conv4_1 = bn(name = "conv4_1_bn")(conv4_1)
    conv4_2 = Conv2D(256, (3, 3), padding = "same", activation='relu', kernel_regularizer=regularizers.l2(l2_lambda), name = "conv4_2")(conv4_1)
    conv4_2 = bn(name = "conv4_2_bn")(conv4_2)
    pool4 = MaxPooling2D(name = "pool4")(conv4_2)
    drop4 = Dropout(dropout_rate)(pool4)  

    conv5_1 = Conv2D(512, (3, 3), padding = "same", activation='relu', kernel_regularizer=regularizers.l2(l2_lambda), name = "conv5_1")(pool4)
    conv5_1 = bn(name = "conv5_1_bn")(conv5_1)
    conv5_2 = Conv2D(512, (3, 3), padding = "same", activation='relu', kernel_regularizer=regularizers.l2(l2_lambda), name = "conv5_2")(conv5_1)
    conv5_2 = bn(name = "conv5_2_bn")(conv5_2)

    # Decoder
    upconv6 = Conv2DTranspose(256,(2, 2), strides=(2, 2), padding='same')(conv5_2)
    upconv6 = Dropout(dropout_rate)(upconv6)
    concat6 = concatenate([conv4_2, upconv6], name = "concat6")
    conv6_1 = Conv2D(256, (3, 3), padding = "same", kernel_regularizer=regularizers.l2(l2_lambda), name = "conv6_1")(concat6)
    conv6_1 = bn(name = "conv6_1_bn")(conv6_1)
    conv6_2 = Conv2D(256, (3, 3), padding = "same", kernel_regularizer=regularizers.l2(l2_lambda), name = "conv6_2")(conv6_1)
    conv6_2 = bn(name = "conv6_2_bn")(conv6_2)

    upconv7 = Conv2DTranspose(128,(2, 2), strides=(2, 2), padding='same')(conv6_2)
    upconv7 = Dropout(dropout_rate)(upconv7)
    concat7 = concatenate([conv3_2, upconv7], name = "concat7")
    conv7_1 = Conv2D(128, (3, 3), padding = "same", kernel_regularizer=regularizers.l2(l2_lambda), name = "conv7_1")(concat7)
    conv7_1 = bn(name = "conv7_1_bn")(conv7_1)
    conv7_2 = Conv2D(128, (3, 3), padding = "same", kernel_regularizer=regularizers.l2(l2_lambda), name = "conv7_2")(conv7_1)
    conv7_2 = bn(name = "conv7_2_bn")(conv7_2)

    upconv8 = Conv2DTranspose(64,(2, 2), strides=(2, 2), padding='same')(conv7_2)
    upconv8 = Dropout(dropout_rate)(upconv8)
    concat8 = concatenate([conv2_2, upconv8], name = "concat8")
    conv8_1 = Conv2D(64, (3, 3), padding = "same", kernel_regularizer=regularizers.l2(l2_lambda), name = "conv8_1")(concat8)
    conv8_1 = bn(name = "conv8_1_bn")(conv8_1)
    conv8_2 = Conv2D(64, (3, 3), padding = "same", kernel_regularizer=regularizers.l2(l2_lambda), name = "conv8_2")(conv8_1)
    conv8_2 = bn(name = "conv8_2_bn")(conv8_2)

    upconv9 = Conv2DTranspose(32,(2, 2), strides=(2, 2), padding='same')(conv8_2)
    upconv9 = Dropout(dropout_rate)(upconv9)
    concat9 = concatenate([conv1_2, upconv9], name = "concat9")
    conv9_1 = Conv2D(32, (3, 3), padding = "same", kernel_regularizer=regularizers.l2(l2_lambda), name = "conv9_1")(concat9)
    conv9_1 = bn(name = "conv9_1_bn")(conv9_1)
    conv9_2 = Conv2D(32, (3, 3), padding = "same", kernel_regularizer=regularizers.l2(l2_lambda), name = "conv9_2")(conv9_1)
    conv9_2 = bn(name = "conv9_2_bn")(conv9_2)
    dropout = Dropout(dropout_rate)(conv9_2)
   
    up6 = UpSampling2D(size=(8,8))(concat6)
    up7 = UpSampling2D(size=(4,4))(concat7)
    up8 = UpSampling2D(size=(2,2))(concat8)
    concate1 = concatenate([up6,up7,up8,dropout])

    conv10 = Conv2D(1, (1, 1), padding = "same", activation = 'sigmoid', name = "conv10")(concate1)


    model = Model(input, conv10)

    return model

In [ ]:
def convolution_block(x, filters, size, strides=(1,1), padding='same', activation=True):
    x = Conv2D(filters, size, strides=strides, padding=padding)(x)
    x = BatchNormalization()(x)
    if activation == True:
        x = LeakyReLU(alpha=0.1)(x)
    return x

def residual_block(blockInput, num_filters=16):
    x = LeakyReLU(alpha=0.1)(blockInput)
    x = BatchNormalization()(x)
    blockInput = BatchNormalization()(blockInput)
    x = convolution_block(x, num_filters, (3,3) )
    x = convolution_block(x, num_filters, (3,3), activation=False)
    x = Add()([x, blockInput])
    return x



def UXception(input_shape=(None, None, 3),dropout_rate=0.5):

    backbone = Xception(input_shape=input_shape,weights='imagenet',include_top=False)
    input = backbone.input
    start_neurons = 16

    conv4 = backbone.layers[121].output
    conv4 = LeakyReLU(alpha=0.1)(conv4)
    pool4 = MaxPooling2D((2, 2))(conv4)
    pool4 = Dropout(dropout_rate)(pool4)
    
     # Middle
    convm = Conv2D(start_neurons * 32, (3, 3), activation=None, padding="same")(pool4)
    convm = residual_block(convm,start_neurons * 32)
    convm = residual_block(convm,start_neurons * 32)
    convm = LeakyReLU(alpha=0.1)(convm)
    
    deconv4 = Conv2DTranspose(start_neurons * 16, (3, 3), strides=(2, 2), padding="same")(convm)
    uconv4 = concatenate([deconv4, conv4])
    uconv4 = Dropout(dropout_rate)(uconv4)
    
    uconv4 = Conv2D(start_neurons * 16, (3, 3), activation=None, padding="same")(uconv4)
    uconv4 = residual_block(uconv4,start_neurons * 16)
    uconv4 = residual_block(uconv4,start_neurons * 16)
    uconv4 = LeakyReLU(alpha=0.1)(uconv4)
    
    deconv3 = Conv2DTranspose(start_neurons * 8, (3, 3), strides=(2, 2), padding="same")(uconv4)
    conv3 = backbone.layers[31].output
    uconv3 = concatenate([deconv3, conv3])    
    uconv3 = Dropout(dropout_rate)(uconv3)
    
    uconv3 = Conv2D(start_neurons * 8, (3, 3), activation=None, padding="same")(uconv3)
    uconv3 = residual_block(uconv3,start_neurons * 8)
    uconv3 = residual_block(uconv3,start_neurons * 8)
    uconv3 = LeakyReLU(alpha=0.1)(uconv3)

    deconv2 = Conv2DTranspose(start_neurons * 4, (3, 3), strides=(2, 2), padding="same")(uconv3)
    conv2 = backbone.layers[21].output
    conv2 = ZeroPadding2D(((1,0),(1,0)))(conv2)
    uconv2 = concatenate([deconv2, conv2])
        
    uconv2 = Dropout(0.1)(uconv2)
    uconv2 = Conv2D(start_neurons * 4, (3, 3), activation=None, padding="same")(uconv2)
    uconv2 = residual_block(uconv2,start_neurons * 4)
    uconv2 = residual_block(uconv2,start_neurons * 4)
    uconv2 = LeakyReLU(alpha=0.1)(uconv2)
    
    deconv1 = Conv2DTranspose(start_neurons * 2, (3, 3), strides=(2, 2), padding="same")(uconv2)
    conv1 = backbone.layers[11].output
    conv1 = ZeroPadding2D(((3,0),(3,0)))(conv1)
    uconv1 = concatenate([deconv1, conv1])
    
    uconv1 = Dropout(0.1)(uconv1)
    uconv1 = Conv2D(start_neurons * 2, (3, 3), activation=None, padding="same")(uconv1)
    uconv1 = residual_block(uconv1,start_neurons * 2)
    uconv1 = residual_block(uconv1,start_neurons * 2)
    uconv1 = LeakyReLU(alpha=0.1)(uconv1)
    
    uconv0 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="same")(uconv1)   
    uconv0 = Dropout(dropout_rate)(uconv0)
    uconv0 = Conv2D(start_neurons * 1, (3, 3), activation=None, padding="same")(uconv0)
    uconv0 = residual_block(uconv0,start_neurons * 1)
    uconv0 = residual_block(uconv0,start_neurons * 1)
    uconv0 = LeakyReLU(alpha=0.1)(uconv0)
    
    uconv0 = Dropout(dropout_rate/2)(uconv0)
    output_layer = Conv2D(1, (1,1), padding="same", activation="sigmoid")(uconv0)    
    
    model = Model(input, output_layer)
    model.name = 'u-xception'

    return model

In [ ]:
from keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='val_dice_coef', factor=0.1,
                              patience=7, min_lr=0.0000001, verbose=1, mode='max')

In [ ]:
model = UXception(input_shape=(768,768,3))
adam = Adam(lr = 0.001)
model.compile(optimizer = adam, loss = 'binary_crossentropy', metrics = [dice_coef])

In [ ]:
#@title Old Unet
input_shape = [512, 512, 1]
dropout_rate = 0.3
l2_lambda = 0.0002

model = u_net(input_shape, dropout_rate, l2_lambda)
adam = Adam(lr = 0.0001)
model.compile(optimizer = adam, loss = dice_loss, metrics = [dice_coef])

In [ ]:
checkpoint = ModelCheckpoint('/content/gdrive/My Drive/Colab Data/Weights/PositiveSampleXceptionUnet_768_rohit17.hdf5', monitor='val_dice_coef', verbose=1, save_best_only=True, mode='max')
#callbacks = [checkpoint,dice_call1]
callbacks = [checkpoint, reduce_lr]

In [ ]:
model.load_weights('/content/gdrive/My Drive/Colab Data/Weights/PositiveSampleXceptionUnet_768.hdf5')

In [ ]:
model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=60, batch_size=1, callbacks=callbacks, verbose=1, shuffle=True)

In [ ]:
model.load_weights('/content/gdrive/My Drive/Colab Data/Weights/PositiveSampleXceptionUnet.hdf5')

In [ ]:
del X_train
del Y_train

In [ ]:
l=np.asarray(X_test, dtype=np.uint8)

In [ ]:
plt.imshow(l[2], cmap='gray')

In [ ]:
x = []
for i in Y_train:
    i = cv2.resize(i,(512,512))
    i = cv2.cvtColor(i, cv2.COLOR_BGR2GRAY)
    #i = np.reshape(i,(512,512,1))
    x.append(i)
x = np.array(x)
x.shape

In [ ]:
x = np.reshape(x,(8245,512,512,1))
x.shape

In [ ]:
np.save('/content/gdrive/My Drive/Colab Data/y_train_int8_1channel_segmentation.npy', x)

In [ ]:
y_pred=model.predict(X_test, verbose=1)

In [ ]:
np.max(y_pred[0])

In [ ]:
plt.imshow(y_pred[1][:,:,0], cmap='gray')

In [ ]:
plt.imshow(y[2][:,:,0], cmap='gray')

In [ ]:
test_csv=pd.read_csv('/content/gdrive/My Drive/Colab Data/SIIM_test.csv')

In [ ]:
predictions=np.around(y_pred)

In [ ]:
predictions=predictions*255

In [ ]:
predictions=predictions.astype('uint8')

In [ ]:
np.max(predictions)

In [ ]:
plt.imshow(predictions[2][:,:,0], cmap='gray')

In [ ]:
def mask2rle(img, width, height):
    rle = []
    lastColor = 0;
    currentPixel = 0;
    runStart = -1;
    runLength = 0;

    for x in range(width):
        for y in range(height):
            currentColor = img[x][y]
            if currentColor != lastColor:
                if currentColor == 255:
                    runStart = currentPixel;
                    runLength = 1;
                else:
                    rle.append(str(runStart));
                    rle.append(str(runLength));
                    runStart = -1;
                    runLength = 0;
                    currentPixel = 0;
            elif runStart > -1:
                runLength += 1
            lastColor = currentColor;
            currentPixel+=1;

    return " ".join(rle)

In [ ]:
test_csv=test_csv.drop(columns='Unnamed: 0')
test_csv.columns

In [ ]:
image_encoded={}
for index, image in enumerate(predictions):
    if np.max(image)==0:
        image_encoded[test_csv['0'][index]]=-1
        
    else:
        image=np.reshape(image, (256,256))
        image_encoded[test_csv['0'][index]]=mask2rle(image, 256, 256)
    print (index)

In [ ]:
df=pd.DataFrame.from_dict(image_encoded, orient='index')

In [ ]:
df_new = pd.DataFrame()

In [ ]:
list_of_images=[]
for i in test_csv['0']:
    a=i.split('/')[-1]
    a=a.split('.png')
    list_of_images.append(a[0])
print (image_encoded)

In [ ]:
df_new['ImageId']=list_of_images

In [ ]:
df_new['EncodedPixels']=image_encoded.values()

In [ ]:
df_new.to_csv('/content/gdrive/My Drive/Colab Data/submission.csv', index=False)

In [ ]:
df_new

In [ ]:
l=np.asarray(X_test, dtype=np.uint8)
plt.imshow(l[0][:,:,0], cmap='gray')

In [ ]:
l=np.asarray(Y_test, dtype=np.uint8)
plt.imshow(l[0][:,:,0], cmap='gray')

In [ ]:
np.max(Y_test)

In [ ]:
final = X_train * y

In [ ]:
final.shape

In [ ]:
plt.imshow(final[2][:,:,0],cmap='gray')

In [ ]:
np.save('/content/gdrive/My Drive/Colab Data/X_train_segmented_lungs.npy', final)

In [ ]:
q=np.asarray(Y_test, dtype='uint8')

In [ ]:
plt.imshow(q[2])

In [ ]:
u=np.load('./data/Unet/unet_yval_for_channel3.npy', mmap_mode='r+')

In [ ]:
u.shape

In [ ]:
q=np.asarray(u, dtype='uint8')

In [ ]:
q.dtype

In [ ]:
plt.imshow(q[2][:,:,0],cmap='gray')